# **WEB SCRAPING ASSIGNMENT - 3**
Submitted by DARSHIK A S

In [ ]:
### import all required libraries
import os
import time
import shutil
import pandas as pd
from selenium import webdriver
import requests
from requests.exceptions import Timeout, SSLError
from selenium.common.exceptions import NoSuchElementException

In [ ]:
### Load webdriver
driver = webdriver.Chrome('chromedriver.exe')

## **1) Search all the product under a particular product vertical from www.amazon.in**

In [ ]:
### Function to search product vertical
def search_product_in_amazon():
    
    ### Input product name from the user
    search_input = input('Enter the product name : ')
    
    ### Go to amazon webpage
    driver.get('http://www.amazon.in/')
    
    ### find search field elements
    search_field = driver.find_element_by_id('twotabsearchtextbox')
    search_button = driver.find_element_by_id('nav-search-submit-button')

    ### search product
    search_field.clear()
    search_field.send_keys(search_input)
    search_button.click()

In [ ]:
### search for any product
search_product_in_amazon()

Enter the product name : guitar


## **2) To scrape the details of each product listed in first 3 pages**

In [ ]:
### Get current window handle
window_handle = driver.current_window_handle

In [ ]:
### Handles exception when product data is not found
def exception_handler(func):
    def inner_func():
        try:
            return func()
        except NoSuchElementException:
            return '-'
    return inner_func

### Handles exception when next page don't exist
def page_exception_handler(func):
    def inner_func():
        try:
            time.sleep(5)
            func()
            time.sleep(5)
        except NoSuchElementException:
            print("No next page")
            pass
    return inner_func

In [ ]:
### Functions to scrape product details from amazon

### Scrape Brand name
@exception_handler
def get_brand_name():
    return driver.find_element_by_id('bylineInfo').text.strip()

### Scrape Product Name
@exception_handler
def get_product_name():
    return driver.find_element_by_id('productTitle').text.strip()

### Scrape Ratings
@exception_handler
def get_rating():
    return driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']").text.strip()

### Scrape No. of Ratings
@exception_handler
def get_no_of_ratings():
    return driver.find_element_by_id('acrCustomerReviewText').text.strip()

### Scrape Price
@exception_handler
def get_price():
    return driver.find_element_by_xpath("//span[contains(@id,'priceblock_')]").text.strip()

### Scrape Return and exchange
@exception_handler
def get_return_exchange():
    return driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']//a").text.strip()

### Scrape Expected delivery
@exception_handler
def get_expected_delivery():
    return driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b").text.strip()

### Scrape Availablility
@exception_handler
def get_availability():
    return driver.find_element_by_xpath("//div[@id='availability']/span").text.strip()

### Scrape Other details
@exception_handler
def get_other_details():
    return driver.find_element_by_xpath("//div[@id='feature-bullets']/ul").text.strip()

### Scrape Go to next page
@page_exception_handler
def go_to_next_page():
    driver.find_element_by_xpath("//li[@class='a-last']/a").click()

In [ ]:
### Set number of pages to scrape
page_limit = 3
### product url element
product_xpath_url = "//div[contains(@class,'sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16') or contains(@class,'s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20')]//a[@class='a-link-normal a-text-normal']"

In [ ]:
### Create list for storing data
brand_name = []
product_name = []
rating = []
no_of_ratings = []
price = []
return_exchange = []
expected_delivery = []
availability = []
other_details = []
product_url = []

In [ ]:
### loop over each pages
for i in range(page_limit):

    ### Get product URLs
    urls = driver.find_elements_by_xpath(product_xpath_url)

    ### Scrape product data
    for url in urls:
        ### scrape Product url
        product_url.append(url.get_attribute("href")) 
        ### Go to product url
        url.click() 
        ### Switch to product window
        child_window = driver.window_handles[1]
        driver.switch_to.window(child_window)
        ### Scrape all other data
        brand_name.append(get_brand_name())
        product_name.append(get_product_name())
        rating.append(get_rating())
        no_of_ratings.append(get_no_of_ratings())
        price.append(get_price())
        return_exchange.append(get_return_exchange())
        expected_delivery.append(get_expected_delivery())
        availability.append(get_availability())
        other_details.append(get_other_details())
        ### Switch back to parent window
        driver.close()
        driver.switch_to.window(window_handle)
    
    if i < (page_limit-1):
        ### Goto next page
        go_to_next_page()
    else:
        break

In [ ]:
### Create dictionary of the data scraped
data_dict = dict(brand_name=brand_name,
                product_name=product_name,
                rating=rating,
                no_of_ratings=no_of_ratings,
                price=price,
                return_exchange=return_exchange,
                expected_delivery=expected_delivery,
                availability=availability,
                other_details=other_details,
                product_url=product_url)

### Convert into DataFrame
data = pd.DataFrame(data_dict)
data.to_csv('amazon_product_details.csv')
data

,brand_name,product_name,rating,no_of_ratings,price,return_exchange,expected_delivery,availability,other_details,product_url
0,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,3.9 out of 5,283 ratings,"₹ 5,999.00",7 Days Replacement,"Sunday, Feb 28",In stock.,QUALITY STRINGS ---The surface is coated with ...,https://www.amazon.in/gp/slredirect/picassoRed...
1,Brand: Medellin,Medellin MED-BLU-C Linden Wood Acoustic Guitar,3.8 out of 5,254 ratings,"₹ 2,299.00",7 Days Replacement,"Sunday, Feb 28",In stock.,Material: Wood\nColour: Blue\nAcoustic Guitar,https://www.amazon.in/gp/slredirect/picassoRed...
2,Brand: Yamaha Guitar,Yamaha Guitar Mexa Yamaha Guitar F310; Dreadno...,4.3 out of 5,28 ratings,"₹ 11,999.00",7 Days Replacement,March 3 - 4,In stock.,raditional Western Body Back/Side/Neck Materia...,https://www.amazon.in/gp/slredirect/picassoRed...
3,Visit the blueberry Store,"Blueberry 38C, 38"" Acoustic Guitar Kit with Ba...",3.4 out of 5,6 ratings,"₹ 2,695.00",7 Days Replacement,"Tuesday, March 2",Only 2 left in stock.,"Material: Top: linden wood, Side & Back: - lin...",https://www.amazon.in/gp/slredirect/picassoRed...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",3.9 out of 5,"7,308 ratings","₹ 2,397.00",7 Days Replacement,"Monday, March 1",In stock.,"Black Glossy Finish, Number of Frets: 18, Acou...",https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...,...,...
169,Brand: ARCTIC,ARCTIC Vigor Acoustic Guitar package with 40 i...,-,-,"₹ 1,377.00",7 Days Replacement,-,In stock.,Made with the quality wood with fine sides and...,https://www.amazon.in/Vigor-Acoustic-Guitar-pa...
170,Brand: VATTU,VATTU String Acoustic Guitar | Adjustable Tunn...,-,-,₹ 999.00,7 Days Replacement,"Wednesday, March 3",In stock.,Toy Guitar is a Great Creativity for little Ki...,https://www.amazon.in/VATTU-Acoustic-Adjustabl...
171,Brand: PENNYCREEK,PennyCreek 6 Pieces Semiclosed Tuners Tuning P...,3.7 out of 5,14 ratings,₹ 480.00,7 Days Replacement,"Thursday, March 4",In stock.,"Set of 12pcs tuning pegs, these are semiclosed...",https://www.amazon.in/PennyCreek-Semiclosed-Pe...
172,Brand: HKD,HKD SALES String Guitar Children's Musical Ins...,-,-,₹ 748.00,7 Days Replacement,"Wednesday, March 3",In stock.,"Made from high quality abs materials, non-toxi...",https://www.amazon.in/HKD-Childrens-Instrument...


## **3) To scrape google images**

In [ ]:
### images to search
search_inputs = {'fruits',
                 'cars',
                 'Machine Learning'}

### number of images to scrape
limit = 100

In [ ]:
### Function to check URL validity
def is_valid_url(url):
    if 'http://' in url[:9] or 'https://' in url[:9]:
        return True
    else:
        return False

In [ ]:
def download_image_from_url(url):
    global count
    global IMAGE_PATH
    filename = str(count)+'.png'
    ### Download images from the url
    try:
        r = requests.get(url,timeout=10,stream = True)
        if r.status_code == 200:
            with open(os.path.join(IMAGE_PATH,filename),"wb") as image_file: 
                for chunk in r.iter_content(chunk_size=1024): 
                    # writing one chunk at a time to image file 
                    if chunk:
                        image_file.write(chunk)
            count = count + 1
    except (Timeout, SSLError):
        pass

In [ ]:
### Scrape 100 images for each keyword
for search_input in search_inputs:
    
    ### Go to google search page
    driver.get('https://images.google.com/')

    ### find search elements
    search_field = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search_button = driver.find_element_by_xpath("//div[@class='FAuhyb']")

    ### search product
    search_field.clear()
    search_field.send_keys(search_input)
    search_button.click()
    
    count = 0
    
    ### Image path to download images
    IMAGE_PATH = os.path.join('google_images',search_input)
    ### Create folders for each search input
    if os.path.exists(IMAGE_PATH):
        shutil.rmtree(IMAGE_PATH)
    os.makedirs(IMAGE_PATH)
    
    ### Loop until 100 images urls are scraped
    while count < limit:
        
        ### Download image from the first focus element
        if count == 0:

            ### Get to first image
            driver.find_element_by_xpath("//img[contains(@class,'Q4LuWd')]").click()
            
            ### Get image url
            url = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//img[@class='n3VNCb']"))).get_attribute('src')

            ### Check url validity
            if is_valid_url(url):
                download_image_from_url(url)

            ### Seek to next image
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"(//a[@class='gvi3cf']/.)[1]"))).click()
            
        ### Get image url
        url = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH,"//img[@class='n3VNCb']"))).get_attribute('src')
        
        ### Check url validity
        if is_valid_url(url):
            download_image_from_url(url)
        
        if count < limit:
            ### Seek to next image
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,"(//a[@class='gvi3cf']/.)[2]"))).click()
    
    print('Successfully extracted {} urls'.format(search_input))

Successfully extracted cars urls
Successfully extracted fruits urls
Successfully extracted Machine Learning urls


In [ ]:
### Checking the image directories
for i in search_inputs:
    print(i,' : ',len(os.listdir(os.path.join('google_images',i))))

cars  :  100
fruits  :  100
Machine Learning  :  100


## **4) To search for a smartphone on www.flipkart.com**

In [ ]:
### Go to Flipkart webpage
driver.get('https://www.flipkart.com/')

### Get current window handle
window_handle = driver.current_window_handle

### Close login popup
try:
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
except NoSuchElementException:
    pass

In [ ]:
### product to be searched
search_input = 'pixel 4A'

### Find elements in the search field
item_field = driver.find_element_by_class_name('_3704LK')
search_button = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")

In [ ]:
### Enter field values
if search_input != None:
    item_field.send_keys(search_input)
search_button.click()

In [ ]:
### create lists to store data
brand_name = []
name = []
color = []
ram = []
rom = []
prim_cam = []
secnd_cam = []
disp_size = []
disp_res = []
proc = []
proc_core = []
bat_capacity = []
price = []
url = []

In [ ]:
### Functions to scrape product data from the flipkart with exception handling

### Scrape product name
@exception_handler
def get_product_name():
    return driver.find_element_by_xpath("//td[text()='Model Name']/following-sibling::td/ul/li").text.strip()

### Scrape color
@exception_handler
def get_color():
    return driver.find_element_by_xpath("//td[text()='Color']/following-sibling::td/ul/li").text.strip()

### Scrape ram
@exception_handler
def get_ram():
    return driver.find_element_by_xpath("//td[text()='RAM']/following-sibling::td/ul/li").text.strip()

### Scrape rom
@exception_handler
def get_rom():
    return driver.find_element_by_xpath("//td[text()='Internal Storage']/following-sibling::td/ul/li").text.strip()

### Scrape primary camera
@exception_handler
def get_prim_cam():
    return driver.find_element_by_xpath("//td[text()='Primary Camera']/following-sibling::td/ul/li").text.strip()

### Scrape secondary camera
@exception_handler
def get_secnd_cam():
    return driver.find_element_by_xpath("//td[text()='Secondary Camera']/following-sibling::td/ul/li").text.strip()

### Scrape display size
@exception_handler
def get_disp_size():
    return driver.find_element_by_xpath("//td[text()='Display Size']/following-sibling::td/ul/li").text.strip()

### Scrape display resolution
@exception_handler
def get_disp_res():
    return driver.find_element_by_xpath("//td[text()='Resolution']/following-sibling::td/ul/li").text.strip()

### Scrape processor
@exception_handler
def get_proc():
    return driver.find_element_by_xpath("//td[text()='Processor Type']/following-sibling::td/ul/li").text.strip()

### Scrape processor core
@exception_handler
def get_proc_core():
    return driver.find_element_by_xpath("//td[text()='Processor Core']/following-sibling::td/ul/li").text.strip()

### Scrape battery capacity
@exception_handler
def get_bat_capacity():
    return driver.find_element_by_xpath("//td[text()='Battery Capacity']/following-sibling::td/ul/li").text.strip()

In [ ]:
### Find the product tag elements
product_tags = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")

### Loop over each products in the page
for product in product_tags:
    
    ### Go to product page
    product.click()
    
    ### Switch to child window
    child_window = driver.window_handles[1]
    driver.switch_to.window(child_window)
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']").click()
    
    ### Scrape product details
    brand_name.append(driver.find_element_by_xpath("//span[@class='B_NuCI']").text.split(' ')[0]) ### Brand name
    name.append(get_product_name())
    color.append(get_color())
    ram.append(get_ram())
    rom.append(get_rom())
    prim_cam.append(get_prim_cam())
    secnd_cam.append(get_secnd_cam())
    disp_size.append(get_disp_size())
    disp_res.append(get_disp_res())
    proc.append(get_proc())
    proc_core.append(get_proc_core())
    bat_capacity.append(get_bat_capacity())
    price.append(driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']").text) ### price
    url.append(driver.current_url)

    ### Switch back to parent window
    driver.close()
    driver.switch_to.window(window_handle)
    
### create dictionary of data scraped
data_dict = dict(
    brand_name=brand_name,
    product_name=name,
    color=color,
    ram=ram,
    rom=rom,
    primary_camera=prim_cam,
    secondary_camera=secnd_cam,
    display_size=disp_size,
    display_res=disp_res,
    processor_type=proc,
    processor_core=proc_core,
    battery_capacity=bat_capacity,
    price=price,
    product_url=url)

In [ ]:
### create csv
data = pd.DataFrame(data_dict)
data.to_csv('flipkart_pixil_4a_results.csv')
data

,brand_name,product_name,color,ram,rom,primary_camera,secondary_camera,display_size,display_res,processor_type,processor_core,battery_capacity,price,product_url
0,Google,Pixel 4a,Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,3140 mAh,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,Redmi,Redmi 9 Prime,Sunrise Flare,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-sunrise...
2,Redmi,Redmi 9 Prime,Matte Black,4 GB,64 GB,13MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080 Pixels,MediaTek Helio G80,Octa Core,5020 mAh,"₹9,499",https://www.flipkart.com/redmi-9-prime-matte-b...
3,Samsung,Galaxy M31,Ocean Blue,6 GB,64 GB,64MP + 8MP + 5MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080$$pixel,Samsung Exynos 9 Octa 9611,Octa Core,6000 mAh,"₹16,276",https://www.flipkart.com/samsung-galaxy-m31-oc...
4,Redmi,Redmi 9 Power,Blazing Blue,4 GB,64 GB,48MP Rear Camera,8MP Front Camera,16.59 cm (6.53 inch),2340 x 1080$$pixel,Qualcomm Snapdragon 662,Octa Core,6000 mAh,"₹10,984",https://www.flipkart.com/redmi-9-power-blazing...
5,Redmi,Redmi Note 9 Pro,Glacier White,4 GB,128 GB,48MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080$$pixel,Qualcomm® Snapdragon™ 720G,Octa Core,5020 mAh,"₹14,285",https://www.flipkart.com/redmi-note-9-pro-glac...
6,Redmi,Redmi 9,Sporty Orange,4 GB,64 GB,13MP + 8MP,8MP Front Camera,16.59 cm (6.53 inch),720 x 1600$$pixel,MediaTek Helio G35,-,5000 mAh,"₹9,274",https://www.flipkart.com/redmi-9-sporty-orange...
7,Samsung,M01 core,Black,1 GB,16 GB,8MP Rear Camera,5MP Front Camera,13.46 cm (5.3 inch),1480 x 720$$pixel,MediaTek | MT6739WW quad core,Octa Core,3000 mAh,"₹4,999",https://www.flipkart.com/samsung-m01-core-blac...
8,Redmi,Redmi Note 9,Aqua Green,6 GB,128 GB,48MP Rear Camera,-,16.59 cm (6.53 inch),2340 x 1080$$pixel,MediaTek Helio G85,Octa Core,5020 mAh,"₹14,620",https://www.flipkart.com/redmi-note-9-aqua-gre...
9,Redmi,Redmi Note 9 Pro,Glacier White,4 GB,64 GB,48MP + 8MP + 5MP + 2MP,16MP Front Camera,16.94 cm (6.67 inch),2400 x 1080 pixel,Qualcomm® Snapdragon™ 720G,Octa Core,5020 mAh,"₹13,399",https://www.flipkart.com/redmi-note-9-pro-glac...


## **5) To scrape geospatial coordinates of a city searched on google maps**

In [ ]:
### Go to google maps webpage
driver.get('https://www.google.com/maps')

In [ ]:
def search_city_google_maps():
    
    ### Enter the city address
    search_input = input('Enter city address : ')
    
    ### Find search field elements
    search_field = driver.find_element_by_id('searchboxinput')
    search_button = driver.find_element_by_id('searchbox-searchbutton')
    
    ### Search the city
    search_field.clear()
    search_field.send_keys(search_input)
    search_button.click()
    
    time.sleep(3)
    
    ### Extract latitude and longitude from the coordinates slug field in the URL
    lat_lon_slug = driver.current_url.split('/')[6][1:].split(',')
    lat = lat_lon_slug[0]
    lon = lat_lon_slug[1]
    
    print('Latitude\t: ',lat)
    print('Longitude\t: ',lon)

In [ ]:
search_city_google_maps()

Enter city address : New York
Latitude	:  40.6971494
Longitude	:  -74.2598766


In [ ]:
search_city_google_maps()

Enter city address : Sardar Vallabhbhai Patel statue, statue of unity
Latitude	:  21.8380184
Longitude	:  73.7168841


## **6) To scrape details of all the funding deals for second quarterfrom trak.in.**

In [ ]:
### Go to trak webpage
driver.get('https://trak.in/')

### Goto funding deals page
driver.find_element_by_xpath("//li[@id='menu-item-51510']").click()

In [ ]:
### Create lists to store data
mon = []
date = []
startup_name = []
industry_vertical = []
sub_vertical = []
city = []
investors_name = []
investment_type = []
amount = []

### months data to scrape from July 2020 to September 2020
months = ['July','August','September']

### Loop to scrape deals in each month
for month in months:
    ### Find table tags
    deal_tags = driver.find_element_by_xpath("//h2[contains(text(),'{}, 2020')]/following-sibling::div//tbody".format(month))
    ### Loop over table and scrape data
    for i in deal_tags.find_elements_by_tag_name('tr'):
        mon.append(month)
        entries = i.find_elements_by_tag_name('td')
        date.append(entries[1].text)
        startup_name.append(entries[2].text)
        industry_vertical.append(entries[3].text)
        sub_vertical.append(entries[4].text)
        city.append(entries[5].text)
        investors_name.append(entries[6].text)
        investment_type.append(entries[7].text)
        amount.append(entries[8].text)
        
### Create dictionary of data
data_dict = dict(
    month=mon,
    date=date,
    startup_name=startup_name,
    industry_vertical=industry_vertical,
    sub_vertical=sub_vertical,
    city=city,
    investors_name=investors_name,
    investment_type=investment_type,
    amount=amount
    )

In [ ]:
### Create csv of the data scraped
data = pd.DataFrame(data_dict)
data.to_csv('trak_funding_deals_july_2020_to_september_2020.csv')
data

,month,date,startup_name,industry_vertical,sub_vertical,city,investors_name,investment_type,amount
0,July,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,July,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,July,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,July,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,July,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,July,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,July,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,July,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,July,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,July,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


## **7) To scrape all the available details of top 10 gaming laptops from digit.in**

In [ ]:
### Go to digit webpage
driver.get('https://www.digit.in/')

In [ ]:
### Goto top gaming laptops page
driver.find_element_by_xpath("//div[@class='listing_container']/ul/li/a[contains(text(),'Gaming Laptops')]").click()

In [ ]:
### Create dictionary to store data
data_dict = {
    'Name':[],
    'Seller':[],
    'Price':[],
    'OS':[],
    'Display':[],
    'Processor':[],
    'Memory':[],
    'Weight':[],
    'Dimension':[],
    'Graphics Processor':[],
    'Description':[],
    'Pros':[],
    'Cons':[]
}

In [ ]:
### Scrape product name, seller name and price of the laptop
for i in driver.find_elements_by_xpath("//table[@id='summtable']//tr")[1:]:
    td_tag = i.find_elements_by_tag_name('td')
    data_dict['Name'].append(td_tag[0].text)
    data_dict['Seller'].append(td_tag[1].text)
    data_dict['Price'].append(td_tag[2].text)
    

### Laptops features to be extracted    
keys = ['OS','Display','Processor','Memory','Weight','Dimension','Graphics Processor']

### Loop over key features
for key in keys:
    for i in driver.find_elements_by_xpath("//td[text()='{}']/following-sibling::td[2]".format(key)):
        data_dict[key].append(i.text)
        
### Loop over and scrape description, pros and cons
for i in driver.find_elements_by_xpath("//div[@class='Section-center']"):
    data_dict['Description'].append(i.find_element_by_tag_name('p').text)
    try:
        data_dict['Pros'].append(i.find_element_by_xpath("div/div[@class='pros_container']/ul").text)
        data_dict['Cons'].append(i.find_element_by_xpath("div/div[@class='pros_container cons']/ul").text)
    except NoSuchElementException:
        data_dict['Pros'].append('-')
        data_dict['Cons'].append('-')

In [ ]:
### Create csv
data = pd.DataFrame(data_dict)
data.to_csv('digit_top_10_best_gaming_laptops.csv')
data

,Name,Seller,Price,OS,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Description,Pros,Cons
0,MSI GT76 TITAN DT 9SG,N/A,₹379990,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,Sporting a desktop graphics Intel Core i9-9900...,-,-
1,Alienware 17 Area-51m,amazon,₹489909,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,The Alienware Area-51m was certainly the first...,-,-
2,HP Omen 15 2020,amazon,₹114990,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,The new HP Omen 15 comes in both 10th generati...,Impressive thermals.\nPerforms well for gamers...,The lid wobbles and flexes.\nNvidia GTX 1650Ti...
3,Asus Zephyrus G14,amazon,₹164990,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,The Asus Zephyrus G14 is a first-of-its-kind g...,Impressive performance for the form-factor.\nA...,Heat makes the keyboard unusable.\nUses a mid-...
4,Lenovo Legion Y540,flipkart,₹79990,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,The Lenovo Legion Y540 is powered by an Intel ...,"Sturdy build, clean look\nTop performance (inc...",Abysmally short battery life\nOddly placed key...
5,Asus ROG Zephyrus G GA502,flipkart,₹79990,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,The Ryzen 7 3750H powered Asus ROG Zephyrus G ...,Noteworthy performance\nComfortable keyboard\n...,Noisy fans can distract\nDisplay could use mor...
6,Asus ROG Zephyrus S GX531,amazon,₹146000,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),The Asus Zephyrus S (GX531) manages to get a f...,-,-
7,MSI GT83VR 7RE Titan SLI,N/A,₹349990,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3.5GHz,1.5 TB SATA/64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070,MSI does not have a dual GTX 1080 gaming lapto...,-,-
8,ASUS ROG ZEPHYRUS DUO 15,amazon,₹259990,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,The machine is powered by an Intel Core i7-108...,-,-
9,Dell G3,amazon,₹61990,Windows 10 Home Plus,15.6 MP | NA,8th Generation Intel® Core™ i7-8750H | NA,256 + 1 TB SSD + SATA/16GB DDR4,2.53,22.7 X 380 X 258,NVIDIA® GeForce® GTX,The Dell G3 is a capable entry-level gaming la...,Seriously fast fingerprint scanner\nSpeakers a...,Bulky at 2.53 kilogrammes\nKeys lack sufficien...
